In [51]:
import sys
import os
import pandas as pd
import numpy as np
import biosignalsnotebooks as bsnb
import biosignalsnotebooks.signal_samples as bsnb_ss
import re
import pickle
import scipy as sp
import datetime
import importlib


In [185]:
import get_features
importlib.reload(get_features)

<module 'get_features' from '/Users/htr365/Documents/Side_Projects/09_founding_lab/amanda_johanna/quantified_self.nosync/pipeline/feature_extraction/get_features.py'>

In [188]:
def load_dataset_group(folder_emg,survey_file, person, outcomes):
    #date_string_list = [date.strftime("%Y-%m-%d") for date in group_set]
    # load class output
    survey = pd.read_csv(survey_file)
    survey = survey[survey['person']==person]
    loaded = list()
    shapes =list()
    y= list()
    for date in survey['date']:
        #emg_am_name = date + '_AM.pickle'
        emg_pm_name = date + '_PM.pickle'
        try:
            
            emg_pm = pd.read_pickle(folder_emg + emg_pm_name)
            print(emg_pm_name)

            # get outcome variable(s)
            y.append(date)
            loaded.append(emg_pm)
        except:
            emg_pm = np.nan
            print("Couldn't load:", emg_pm_name)


    # get most frequent input shape
    #standard_shape = pd.Series(shapes).mode()
    #entries_correct = np.array([i for i, x in enumerate(shapes) if x==standard_shape[0]])

    #X = np.dstack([loaded[i] for i in entries_correct])
    #y = np.array([y[i] for i in entries_correct])
    return loaded,y


In [477]:

folder_emg = '/Users/htr365/Documents/Side_Projects/09_founding_lab/amanda_johanna/quantified_self.nosync/pipeline/data_pre_processed/'
person = 'Johanna'
outcomes = 'hooper_fatigue_level'
survey_file = '/Users/htr365/Documents/Side_Projects/09_founding_lab/amanda_johanna/quantified_self.nosync/pipeline/survey_data/pre_processed_survey_features.csv'
survey = pd.read_csv(survey_file)
X,y =load_dataset_group(folder_emg,survey_file, person, outcomes)

2024-06-24_PM.pickle
2024-06-25_PM.pickle
Couldn't load: 2024-06-26_PM.pickle
2024-06-27_PM.pickle
2024-06-28_PM.pickle
2024-06-29_PM.pickle
2024-06-30_PM.pickle
2024-07-01_PM.pickle
2024-07-02_PM.pickle
2024-07-03_PM.pickle
2024-07-04_PM.pickle
2024-07-05_PM.pickle
2024-07-06_PM.pickle
2024-07-07_PM.pickle
2024-07-08_PM.pickle
2024-07-09_PM.pickle
2024-07-10_PM.pickle
2024-07-11_PM.pickle
2024-07-12_PM.pickle
2024-07-13_PM.pickle
2024-07-14_PM.pickle
2024-07-15_PM.pickle
2024-07-16_PM.pickle
2024-07-17_PM.pickle
2024-07-18_PM.pickle
2024-07-19_PM.pickle
2024-07-20_PM.pickle


In [462]:
emg_data = pd.DataFrame()
for i in range(len(X)):
    # artifically create time range
    #emg_features = get_features.get_emg_features(X[i][(X[i]['time'] > 31)*(X[i]['time']<36)])[0].T
    emg_features  = X[i][(X[i]['time'] > 31)*(X[i]['time']<36)]
    sr = 1000
    low_pass = 1
    low_pass = low_pass/(sr/2)
    b2, a2 = sp.signal.butter(4, low_pass, btype='lowpass')

    emg_features['emg_envelope'] = sp.signal.filtfilt(b2, a2, emg_features['signal_emg'])


    time = pd.date_range(start=y[i], end=pd.to_datetime(y[i])+datetime.timedelta(days=1), periods=emg_features.shape[0])
    emg_features =  pd.concat([pd.Series(time).rename('date'),emg_features[['signal_emg','emg_envelope','signal_mmg']].reset_index()],axis=1)
    emg_data = pd.concat([emg_data, emg_features])


/var/folders/m2/yy15wc4d3zqdtsvvvggmkn780000gp/T/ipykernel_1423/3628256550.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emg_features['emg_envelope'] = sp.signal.filtfilt(b2, a2, emg_features['signal_emg'])
/var/folders/m2/yy15wc4d3zqdtsvvvggmkn780000gp/T/ipykernel_1423/3628256550.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  emg_features['emg_envelope'] = sp.signal.filtfilt(b2, a2, emg_features['signal_emg'])
/var/folders/m2/yy15wc4d3zqdtsvvvggmkn780000gp/T/ipykernel_1423/3628256550.py:11: Se

In [428]:
emg_data

,date,index,signal_emg,emg_envelope,signal_mmg
0,2024-06-24 00:00:00.000000000,31000,38.610144,36.939981,1.059602
1,2024-06-24 00:00:17.283456691,31001,16.766941,36.951904,1.109868
2,2024-06-24 00:00:34.566913382,31002,48.720320,36.963851,1.175209
3,2024-06-24 00:00:51.850370074,31003,101.506986,36.975821,1.256688
4,2024-06-24 00:01:09.133826765,31004,133.330333,36.987816,1.353021
...,...,...,...,...,...
4995,2024-07-20 23:58:50.866173234,35995,28.510107,54.964184,1.263821
4996,2024-07-20 23:59:08.149629925,35996,34.383133,54.964184,1.216329
4997,2024-07-20 23:59:25.433086617,35997,21.317767,54.964184,1.167924
4998,2024-07-20 23:59:42.716543308,35998,11.429565,54.964184,1.122230


In [463]:
colMax = np.max(emg_data[['signal_emg','emg_envelope','signal_mmg']],axis=0)

emg_data[['signal_emg','emg_envelope','signal_mmg']] = emg_data[['signal_emg','emg_envelope','signal_mmg']] / colMax

In [464]:
colMax = np.max(emg_data[['WAMP', 'MYOP', 'FR', 'MNP', 'TP', 'MNF', 'MDF', 'PKF', 'WENT']],axis=0)

emg_data[['VAR', 'RMS', 'IEMG', 'MAV', 'LOG', 'WL', 'ACC', 'DASDV',
       'WAMP', 'MYOP', 'FR', 'MNP', 'TP', 'MNF', 'MDF', 'PKF', 'WENT']] = emg_data[['VAR', 'RMS', 'IEMG', 'MAV', 'LOG', 'WL', 'ACC', 'DASDV',
      'WAMP', 'MYOP', 'FR', 'MNP', 'TP', 'MNF', 'MDF', 'PKF', 'WENT']] / colMax

KeyError: "None of [Index(['WAMP', 'MYOP', 'FR', 'MNP', 'TP', 'MNF', 'MDF', 'PKF', 'WENT'], dtype='object')] are in the [columns]"

In [478]:
df_johanna = survey[survey['person']=='Johanna']
#.merge(df,how='left',left_on='date',right_on='date')

In [479]:
def nth_col_with_one(row, n):
    count = 0
    for col in ['walking','cycling','ball_sports','weight_lifting','running','yoga','HITT','hiking','manual_labour']:
        if row[col] == 1:
            count += 1
            if count == n:
                return col
    return None

In [480]:
for n in range(1, 3 + 1):
    col_name = f'{n}th_col_with_one'
    df_johanna[col_name] = df_johanna[['walking','cycling','ball_sports','weight_lifting','running','hiking','yoga','HITT','manual_labour']].apply(nth_col_with_one, axis=1, n=n)


/var/folders/m2/yy15wc4d3zqdtsvvvggmkn780000gp/T/ipykernel_1423/2206050293.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_johanna[col_name] = df_johanna[['walking','cycling','ball_sports','weight_lifting','running','hiking','yoga','HITT','manual_labour']].apply(nth_col_with_one, axis=1, n=n)
/var/folders/m2/yy15wc4d3zqdtsvvvggmkn780000gp/T/ipykernel_1423/2206050293.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_johanna[col_name] = df_johanna[['walking','cycling','ball_sports','weight_lifting

In [481]:
df_johanna = df_johanna.dropna(subset=['fatigue_expected'])

In [482]:
colMeans = np.mean(df_johanna[['cis_subjective_fatigue', 'cis_concentration','cis_physical_activity','cis_motivation']],axis=0)
df_johanna[['cis_subjective_fatigue', 'cis_concentration','cis_physical_activity','cis_motivation']] = \
       np.array((df_johanna[['cis_subjective_fatigue', 'cis_concentration','cis_physical_activity','cis_motivation']] - colMeans)/colMeans)

df_johanna['fatigue_expected'] = (df_johanna['fatigue_expected']-colMeans['cis_subjective_fatigue'])/colMeans['cis_subjective_fatigue']

In [483]:
df_johanna[['cis_subjective_fatigue', 'cis_concentration','cis_physical_activity','cis_motivation','fatigue_expected']] = df_johanna[['cis_subjective_fatigue', 'cis_concentration','cis_physical_activity','cis_motivation','fatigue_expected']].apply(lambda x: x-min(x))

In [472]:
df_johanna[['cis_subjective_fatigue', 'cis_concentration','cis_physical_activity','cis_motivation','fatigue_expected']] 

,cis_subjective_fatigue,cis_concentration,cis_physical_activity,cis_motivation,fatigue_expected
8,0.300000,0.490654,0.190045,0.180344,0.800000
9,0.233333,0.245327,0.000000,0.000000,0.800000
10,0.266667,0.294393,0.095023,0.240458,0.800000
11,0.066667,0.294393,0.665158,0.300573,0.533333
12,0.233333,0.196262,0.380090,0.420802,0.000000
13,0.000000,0.000000,0.285068,0.781489,0.266667
14,0.133333,0.147196,0.000000,1.202290,0.000000
15,0.133333,0.147196,0.000000,1.041985,0.000000
17,0.233333,0.294393,0.095023,0.781489,0.000000
19,0.033333,0.294393,0.095023,0.841603,0.000000


In [484]:
df_johanna['muscles_sore_prediction']= df_johanna['muscles_sore_expected'].shift(1)
df_johanna['muscles_tired_prediction']= df_johanna['muscles_tired_expected'].shift(1)
df_johanna['fatigue_prediction']= df_johanna['fatigue_expected'].shift(1)

In [377]:
df_johanna['date']

Series([], Name: date, dtype: object)

In [485]:
df_johanna['date'] = df_johanna['date'] +' 00:00:00.000000000'

In [350]:
df_johanna['date'] 

Series([], Name: date, dtype: object)

In [431]:
emg_data = emg_data[emg_data['date'] >= df_johanna['date'].iloc[0]]

In [486]:
df_johanna['sleep_hours'] =pd.to_numeric(df_johanna['sleep_hours'].str[:4].str.replace(':','.')).round().astype(int)

In [487]:
df_johanna.to_csv('/Users/htr365/Documents/Side_Projects/09_founding_lab/amanda_johanna/quantified_self.nosync/website/docs/data/data_johanna.csv')

In [491]:
df_johanna['date'].max()+datetime.timedelta(days=1)

'2024-07-20 00:00:00.000000000'

In [441]:
survey.columns

Index(['Unnamed: 0', 'person', 'date', 'cis_1', 'cis_2', 'cis_3', 'cis_4',
       'cis_5', 'cis_6', 'cis_7', 'cis_8', 'cis_9', 'cis_10', 'cis_11',
       'cis_12', 'cis_13', 'cis_14', 'cis_15', 'cis_16', 'cis_17', 'cis_18',
       'cis_19', 'cis_20', 'hooper_muscles_sore', 'extra_muscles_tired',
       'hooper_sleep_quality', 'hooper_fatigue_level', 'hooper_stress_level',
       'sleep_hours', 'coffee_count', 'physical_activity', 'step_count',
       'alcohol', 'fatigue_expected', 'muscles_sore_expected',
       'muscles_tired_expected', 'cis_subjective_fatigue', 'cis_concentration',
       'cis_motivation', 'cis_physical_activity', 'sleep_time', 'walking',
       'cycling', 'ball_sports', 'weight_lifting', 'running', 'yoga', 'HITT',
       'manual_labour', 'cis_subjective_fatigue_lead1',
       'hooper_fatigue_level_lead1'],
      dtype='object')

In [432]:
emg_data.to_csv('/Users/htr365/Documents/Side_Projects/09_founding_lab/amanda_johanna/quantified_self.nosync/website/docs/data/emg_johanna.csv')

In [195]:
df_johanna['cis_subjective_fatigue'].m

3.725